In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import os.path

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

%matplotlib inline

In [85]:
datapath = os.path.expanduser('~') + '/meddata/'
dfVar = pd.read_csv(datapath + 'training_variants') #,index_col='ID')


In [86]:
dfVar.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [87]:
file = open(datapath + 'training_text', 'r') 
rawText = file.read() 

In [88]:
rawText[:123]

'ID,Text\n0||Cyclin-dependent kinases (CDKs) regulate a variety of fundamental cellular processes. CDK10 stands out as one of'

In [89]:
lines = rawText.split('\n')[1:]

splitted = [line.split('||') for line in lines]

dfText = pd.DataFrame(splitted, columns=['ID','Text'])



In [90]:
dfText['ln'] = dfText['ID'].map(lambda x: len(x))
dfText = dfText[dfText['ln'] > 0]
dfText.drop('ln',axis=1,inplace=True)


In [91]:
dfText['ID'] = dfText['ID'].map(lambda n:int(n))

In [92]:
dfText.head()

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [93]:
df = pd.merge(dfText, dfVar, on='ID')
df.set_index('ID',drop=True,inplace=True)
df['source'] = 'train'

In [94]:
df.head()

,Text,Gene,Variation,Class,source
ID,,,,,
0,Cyclin-dependent kinases (CDKs) regulate a var...,FAM58A,Truncating Mutations,1,train
1,Abstract Background Non-small cell lung canc...,CBL,W802*,2,train
2,Abstract Background Non-small cell lung canc...,CBL,Q249E,2,train
3,Recent evidence has demonstrated that acquired...,CBL,N454D,3,train
4,Oncogenic mutations in the monomeric Casitas B...,CBL,L399V,4,train


In [95]:
dfVarT = pd.read_csv(datapath + 'test_variants') #,index_col='ID')

In [96]:
dfVarT.head()

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A


In [97]:
fileT = open(datapath + 'test_text', 'r') 
rawTextT = fileT.read() 

In [98]:
rawTextT[:123]

'ID,Text\n0||2. This mutation resulted in a myeloproliferative phenotype, including erythrocytosis, in a murine model of retr'

In [99]:
linesT = rawTextT.split('\n')[1:]

splittedT = [line.split('||') for line in lines]

dfTextT = pd.DataFrame(splittedT, columns=['ID','Text'])

dfTextT['ln'] = dfTextT['ID'].map(lambda x: len(x))
dfTextT = dfTextT[dfTextT['ln'] > 0]
dfTextT.drop('ln',axis=1,inplace=True)
dfTextT['ID'] = dfTextT['ID'].map(lambda n:int(n))

In [100]:
dfT = pd.merge(dfTextT, dfVarT, on='ID')
dfT.set_index('ID',drop=True,inplace=True)
dfT['source'] = 'test'

In [101]:
dfT.head()

,Text,Gene,Variation,source
ID,,,,
0,Cyclin-dependent kinases (CDKs) regulate a var...,ACSL4,R570S,test
1,Abstract Background Non-small cell lung canc...,NAGLU,P521L,test
2,Abstract Background Non-small cell lung canc...,PAH,L333F,test
3,Recent evidence has demonstrated that acquired...,ING1,A148D,test
4,Oncogenic mutations in the monomeric Casitas B...,TMEM216,G77A,test


In [102]:
df=df[['Text','source']]
dfT=dfT[['Text','source']]

In [103]:
dfT.head()

,Text,source
ID,,
0,Cyclin-dependent kinases (CDKs) regulate a var...,test
1,Abstract Background Non-small cell lung canc...,test
2,Abstract Background Non-small cell lung canc...,test
3,Recent evidence has demonstrated that acquired...,test
4,Oncogenic mutations in the monomeric Casitas B...,test


In [104]:
data = df.append(dfT)

In [105]:
tfidf = TfidfVectorizer()
data['tfidfFeatures'] = tfidf.fit_transform(data['Text'])

In [106]:
#x.toarray()
#v.get_feature_names()
data.head()

,Text,source,tfidfFeatures
ID,,,
0,Cyclin-dependent kinases (CDKs) regulate a var...,train,"(0, 46477)\t0.230859469076\n (0, 49893)\t0...."
1,Abstract Background Non-small cell lung canc...,train,"(0, 46477)\t0.230859469076\n (0, 49893)\t0...."
2,Abstract Background Non-small cell lung canc...,train,"(0, 46477)\t0.230859469076\n (0, 49893)\t0...."
3,Recent evidence has demonstrated that acquired...,train,"(0, 46477)\t0.230859469076\n (0, 49893)\t0...."
4,Oncogenic mutations in the monomeric Casitas B...,train,"(0, 46477)\t0.230859469076\n (0, 49893)\t0...."
